### SimpleMKL Training

- In scikit-learn we utilized a single kernel (radial basis function) with cross validated bandwidth (via grid search)

- We now want to extend to the case of multiple kernels (linear combination of a basis set)

- Inspiration in our original implementation from here 

     - https://github.com/qintian0321/SimpleMKL_python

In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
from scipy.spatial import distance_matrix

# SimpleMKL optimization objective

### Primal problem 


### Dual Problem

$$\max_\alpha \ \frac{-1}{2} \sum_{i,j} \alpha_i \alpha_j y_i y_j \sum_m d_mK_m(x_i,x_j) +\sum_i \alpha_i$$

s.t. $$\sum_i \alpha_i y_i=0$$ and $$C \geq \alpha_i \geq 0$$



- Coefficient vector $ \alpha$ is importance of observed features on classification problem




### Abstract Kernel Class 

- Define an abstract kernel class wich allows for the computation of polynomial or gaussian kernels of varying order /bandwidth

In [2]:
class Kernel():
    """ Abstract method to compute the kernel matrix under gaussian kernel or polynomial kernel"""
    
    def __init__(self,kernel_type,order=None,bandwidth=None):
        self.kernel_type=kernel_type
        self.order=order
        self.bandwidth=bandwidth
    
    def compute_kernel(self,X):
        
        if self.kernel_type=='linear':
            return np.dot(X,X.T)
        
        
        if self.kernel_type=='gaussian':
            # Compute the distance matrix which is then numerically transformed to gaussian kernel
            scale=distance_matrix(X,X,p=2)
            return np.exp(-0.5*self.bandwidth*(scale**2))
        
        if self.kernel_type=='polynomial':
            return np.dot(X,X.T)**self.order
        

### Compose a linear combination of Kernel Matrices


In [3]:
def compose_kernels(X,kernel_list,weights):
    """ Compute positive linear combination of kernels 
    """
    return np.sum(np.array([weights[ct]*k_i.compute_kernel(X) for ct,k_i in enumerate(kernel_list)]),axis=0)

### Functions for Multiple Kernel Optimization

- Objective is Optimized via Gradient Descent
- Main functionality required

### Compute Dual objective and Duality Gap

In [4]:
def compute_dual(X,y,kernel_list,d_m,constant,compute_gap=True):
    """ Compute dual objective value for single SVM
    """
    kernel=compose_kernels(X,kernel_list,d_m)
    single_kernel=svm.SVC(C=constant,kernel='precomputed')
    single_kernel.fit(kernel,y)
    
    alpha=np.empty(len(y))
    alpha[single_kernel.support_]=np.abs(single_kernel.dual_coef_[0]) 
    alpha[alpha==None]=0

    y_outer=np.outer(y,y)
    
    J=-0.5*np.dot(np.dot(alpha,np.multiply(y_outer,kernel)),alpha.T)+np.sum(alpha)
    
    if compute_gap:
        kernel_eval=[np.dot(np.dot(alpha,np.multiply(y_outer,k_i.compute_kernel(X))),alpha.T) for k_i in kernel_list]
        duality_gap=J-np.sum(alpha)+0.5*np.max(kernel_eval)
        
        return J,duality_gap,alpha
    
    return J

### Compute the Gradient with respect to kernel weight vector $d_M$

In [5]:
def compute_gradient(kernel,X,y_outer,alpha):
    """ Compute gradient of MKL objective closed form ; vector
    """
    kernel_mat=kernel.compute_kernel(X)
    gradient_obj=-0.5*np.dot(np.dot(alpha,np.multiply(y_outer,kernel_mat)),alpha.T)
    
    return gradient_obj

### Compute Direction of Descent respecting equality conditions
- Most optimal direction might violate normality and nonnegative conditons

In [6]:
def descent_direction(d_m,mu,gradient_j,grad_mu):
    """ Compute direction of gradient descent ; vector 
    """
    n=len(d_m)
    D=np.zeros(n)
    ongoing_sum=0
    for index in range(0,n):
        if d_m[index]==0 and gradient_j[index]-grad_mu>0:
            D[index]=0
    
        elif d_m[index]>0 and index!=mu:
            
            grad_m=-gradient_j[index]+grad_mu
            D[index]=grad_m
            ongoing_sum+=grad_m  
        
        else:
            D[index]=0
         
    D[mu]=-ongoing_sum

    return D

### Line Search for Optimal Step Size $gamma_{max}$

In [7]:
def line_search(X,y,kernel_list,D,d_m,gamma_max,disc):
    """ Selects step size to minimize obj value;  
    
        Update from heuristic to exact Armijo's rule 
    """

    if gamma_max==0:
        return gamma_max
    
    # grid of step size begins bigger than 0
    grid=np.arange(0+gamma_max/disc,gamma_max,gamma_max/disc)
    
    min_gamma,min_obj_val=None,10e8
    for gamma_i in grid:
        d_i=d_m+gamma_i*D
        dual_obj_val=compute_dual(X,y,kernel_list,d_i,constant=100,compute_gap=False)
        
        if abs(dual_obj_val)<abs(min_obj_val):
            min_obj_val=dual_obj_val
            min_gamma=gamma_i
        
    
    return min_gamma

### Main Primal-Dual Method

In [8]:
def primal_dual_opt(X,y,m,kernel_type,order,gap=10e-4,inner_tol=10e-1,weight_threshold=0.01,maxouter_iter=100,maxinner_iter=10 ,verbose=True,identical=False):
    """ Computes the optimal weights for the MKL objective using primal dual method
    """
    
    duality_gap=1 # initialize duality gap
    C=0.01 # penalty constant for SVM
    y_outer=np.outer(y,y) # outer product of y for efficiency
    counter=0
    d_m=np.ones(m)/m # initialize weights
    D=np.ones(m) # initialize descent direction
    mu=0 # initialize index of weight to be updated
    line_search_steps=25 # number of steps for line search
    gamma_max=0 # initialize step size
    
    # initialize kernel types
    if kernel_type=='linear':
        if not identical:
            kernel_list=[Kernel(kernel_type=kernel_type,order=1)for i in range(1,order+1) ]
        else:
            kernel_list=[Kernel(kernel_type=kernel_type,order=1)for i in range(0,order+1) ]
    elif kernel_type=='polynomial':
        if not identical:
            kernel_list=[Kernel(kernel_type=kernel_type,order=i)for i in range(1,order+1) ]
        else:
            kernel_list=[Kernel(kernel_type,1) for i in range(1,order+1)]
    elif kernel_type=='gaussian':
        if not identical:
            kernel_list=[Kernel(kernel_type,bandwidth=i) for i in np.linspace(0.1,1,m)] # gamma hyperparam
        else:
            kernel_list=[Kernel(kernel_type,bandwidth=0.5) for i in range(0,m)] 

    else:
        print("Not Valid Kernel Type")
        return
    
    # stopping criteria is duality gap
    while duality_gap>gap and gap>0:
        if counter>maxouter_iter:
            break
        counter+=1

        # compute svm objective
        J_d,duality_gap,alpha=compute_dual(X,y,kernel_list,d_m,C) 
        if verbose:
            print("Duality",duality_gap)

        # gradient wrt each kernel
        gradient_j=[compute_gradient(i,X,y_outer,alpha) for i in kernel_list] 
        if verbose:
            print("Gradient is ",gradient_j)
   

        # max element within d vector
        mu=np.argmax(d_m)
        grad_mu=gradient_j[mu]
        
        # computes normalized descent direction ; satisfies equality constraints 
        D=descent_direction(d_m,mu,gradient_j,grad_mu)
        norm_D=np.sqrt(D.dot(D))
        if norm_D==0.0:
            break
        D=D/norm_D
        if verbose:
            print("Descent Direction is ",D)
        
        # init descent direction update
        J_hat=0
        d_hat=d_m
        D_hat=D
        inner_iter=0
        
        #default zero step size
        gamma_list=[0]
       
        while J_hat<J_d :
            
            if inner_iter>maxinner_iter:
                break
            
            inner_iter+=1 
            
            # indices where descent direction is negative, 
            # if none reached local max, else update by gamma step
            nonzero_D=np.where(D_hat<0)[0]
            if len(nonzero_D)==0:
                gamma_max=0  
            else:
                gamma_max=np.min(-d_hat[nonzero_D]/D_hat[nonzero_D])

            gamma_list.append(gamma_max)
            D=D_hat
            d_m=d_hat
            d_hat=d_m+gamma_max*D
            d_hat[d_hat<weight_threshold]=0
                
            D_hat[mu]=descent_direction(d_hat,mu,gradient_j,grad_mu)[mu]
            J_hat=compute_dual(X,y,kernel_list,d_hat,C,compute_gap=False)
            
        # line search in descent direction
        gamma_max=np.max(gamma_list)  
        gamma_step=line_search(X,y,kernel_list,D,d_m,gamma_max,disc=line_search_steps)
        
       
        d_m=(d_m+gamma_step*D)
        if verbose:
            print("Gamma Max is ",gamma_max)
            print("Gamma Step is ",gamma_step)
        
        # normalize and drop threshold
        d_m[d_m<weight_threshold]=0
        d_m=d_m/np.sum(d_m)
        
        if verbose:
            print("Weights are ",d_m)
       
    if abs(duality_gap)<gap:
            print("Duality Gap Reached of ",duality_gap)
            return d_m,kernel_list
    else:
        print("Max Iterations Reached")
        return d_m,kernel_list
    

### Form optimal Kernel as a linear combination of multiple kernels

In [9]:
def form_optimal_svm(kernel_list,d_m,X,y):
    """ Forms optimal SVM from optimal weights and kernels
    """
    # compute optimal kernel
    K=np.zeros((X.shape[0],X.shape[0]))
    for i in range(len(kernel_list)):
        K+=d_m[i]*kernel_list[i].compute_kernel(X)
    
    # fit svm
    clf=svm.SVC(kernel='precomputed')
    clf.fit(K,y)
    return clf,K

### Test MKL Function on simulated Data

In [10]:
def test_mkl(kernel_type,m,verbose=True,identical=False):
    
    n=200# data set size
    y=np.sign(np.random.uniform(-1,1,size=n)) # sample class labels 1,-1
    x=np.random.rand(n,5) # # sample features

  
    d_m,kernel_list=primal_dual_opt(x,y,m,kernel_type,order=m,verbose=verbose,identical=identical)
    
    print("Optimal Weights are ",np.round(d_m,2))

    optimal_svm,K_composed=form_optimal_svm(kernel_list,d_m,x,y)
    
    # compute accuracy
    print("In Sample Accuracy is ",optimal_svm.score(K_composed,y))


### Basis of polynomial kernels 

In [11]:
test_mkl("polynomial",m=4)

Duality 2012.944590491737
Gradient is  [-639.5244777516392, -1073.3106570989924, -1967.6331405851697, -3910.7488791342494]
Descent Direction is  [-0.81663836  0.07038308  0.21548952  0.53076575]
Gamma Max is  0
Gamma Step is  0
Weights are  [0.25 0.25 0.25 0.25]
Duality 677.0984259780561
Gradient is  [-236.9523631671339, -384.6028120908815, -686.2378406519587, -1338.7289066073997]
Descent Direction is  [-0.81699148  0.07101211  0.21608272  0.52989665]
Gamma Max is  0
Gamma Step is  0
Weights are  [0.25 0.25 0.25 0.25]
Duality 2012.944590491737
Gradient is  [-639.5244777516392, -1073.3106570989924, -1967.6331405851697, -3910.7488791342494]
Descent Direction is  [-0.81663836  0.07038308  0.21548952  0.53076575]
Gamma Max is  0
Gamma Step is  0
Weights are  [0.25 0.25 0.25 0.25]
Duality 2012.944590491737
Gradient is  [-639.5244777516392, -1073.3106570989924, -1967.6331405851697, -3910.7488791342494]
Descent Direction is  [-0.81663836  0.07038308  0.21548952  0.53076575]
Gamma Max is  0
Ga

In [12]:
test_mkl("polynomial",m=4,identical=True)

Duality -4.960055532531732e-17
Gradient is  [-4.695957842086027e-07, -4.695957842086027e-07, -4.695957842086027e-07, -4.695957842086027e-07]
Duality Gap Reached of  -4.960055532531732e-17
Optimal Weights are  [0.25 0.25 0.25 0.25]
In Sample Accuracy is  0.535


### Basis of Gaussian Kernels

In [13]:
test_mkl("gaussian",m=5)

Duality 0.00025997685227917755
Gradient is  [-1.3049625527750345e-05, -7.550206913448329e-05, -0.0001770336848020246, -0.00031029485188502516, -0.0004689411231862858]
Descent Direction is  [-0.8636229   0.05506006  0.14457356  0.26206084  0.40192844]
Gamma Max is  0.23158255671301214
Gamma Step is  0.22231925444449166
Weights are  [0.         0.18875156 0.22283742 0.26757541 0.32083561]
Duality Gap Reached of  0.00025997685227917755
Optimal Weights are  [0.   0.19 0.22 0.27 0.32]
In Sample Accuracy is  0.545


In [14]:
test_mkl("gaussian",m=5,identical=True)

Duality 1.0820337681405334e-16
Gradient is  [-0.0007016689398972014, -0.0007016689398972014, -0.0007016689398972014, -0.0007016689398972014, -0.0007016689398972014]
Duality Gap Reached of  1.0820337681405334e-16
Optimal Weights are  [0.2 0.2 0.2 0.2 0.2]
In Sample Accuracy is  0.56


### Main Results Learned

- Dual objective and differential are similar as objective is linear in d
- Curse of dimensionality; as we increase basis size of kernels; optimizer is increasingly slow
- Gradient will be negative as we are performing gradient descent
- Actual descent direction might be positive to respect equality constraints 
- Choose Gamma Step as a line search between potential values within [0, GammaMax]

